# Figure 3: NIRCam Grism Dispersion Lengths for Module A

***
### Table of Contents

1. [Information](#Information)
2. [Imports](#Imports)
3. [Data](#Data)
4. [Generate the Grism Dispersion Lengths Plot](#Generate-the-Grism-Dispersion-Lengths-Plot)
5. [Issues](#Issues)
6. [About this Notebook](#About-this-Notebook)
***

## Information

#### JDox links: 
* [NIRCam Grisms](https://jwst-docs.stsci.edu/display/JTI/NIRCam+Grisms#NIRCamGrisms-Dispersion)
    * Figure 3. NIRCam grism dispersion lengths for module A

## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from astropy.io import ascii, fits
from astropy.table import Table
from scipy.optimize import fmin
from scipy.interpolate import interp1d
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline

## Data

#### Data Location: 

The data is stored in a NIRCam JDox Box folder here:
[ST-INS-NIRCAM -> JDox -> nircam_grisms](https://stsci.box.com/s/wu9mo54vi957x50rdirlcg9zkkr3xiaw)

#### Load the data

(The next cell assumes you downloaded the data into your ```Users/$(logname)/``` home directory)

In [ ]:
box_directory = "/Users/"+os.environ['LOGNAME']+"/box_data/"   
box_directory

In [ ]:
filenames = []
for fil in filters:
    filenames.append(box_directory+"NIRCam.%s.%s.%s.2nd.sensitivity.fits" % (fil,grism,mod))                  

In [ ]:
filenames

In [ ]:
grism = "R"
mod = "A"
thr = 0.05 # 5% of peak boundaries
filters = ["F250M","F277W","F300M","F322W2","F335M","F356W","F360M","F410M","F430M","F444W","F460M","F480M"]

### Define some convenience functions

In [ ]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]


def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]


def find_mid(w,s,w0,thr=0.05):
    fct = interp1d(w,s,bounds_error=None,fill_value='extrapolate')
    def func(x):
        #print "x:",x
        return np.abs(fct(x)-thr)
    
    res = fmin(func,w0)
    
    return res[0]

In [ ]:
tab = []
for fname in filenames:
    d = fits.open(fname)
    w = d[1].data["WAVELENGTH"]
    s = d[1].data["SENSITIVITY"]
    s = s/np.max(s)
    wmin,wmax = np.min(w),np.max(w)
    vg = w<(wmax+wmin)/2.
    w1 = find_mid(w[vg],s[vg],wmin,thr)
    
    vg = w>(wmax+wmin)/2.
    w2 =  find_mid(w[vg],s[vg],wmax,thr)
#     print('\n')
#     print('Filter,lamda1,lamda2,dispersedpix: '+str([fil,w1,w2,(w2-w1)*1000]))
#     print('\n\n')
    
    tab.append([fil,w1,w2,(w2-w1)*1000])
    
columns=['filter','lambda1','lambda2','npix']
df = pd.DataFrame(tab, columns=columns)
table = Table.from_pandas(df)

In [ ]:
tab2 = []
for fname in filenames:
    d = fits.open(fname)
    w = d[1].data["WAVELENGTH"]
    s = d[1].data["SENSITIVITY"]
    s = s/np.max(s)
    wmin,wmax = np.min(w),np.max(w)
    vg = w<(wmax+wmin)/2.
    w1 = find_mid(w[vg],s[vg],wmin,thr)
    
    vg = w>(wmax+wmin)/2.
    w2 =  find_mid(w[vg],s[vg],wmax,thr)
#     print('\n')
#     print('Filter,lamda1,lamda2,dispersedpix: '+str([fil,w1,w2,(w2-w1)*1000]))
#     print('\n\n')
    
    tab2.append([fil,w1,w2,(w2-w1)*1000])
    
columns=['filter','lambda1','lambda2','npix']
df = pd.DataFrame(tab2, columns=columns)
table2 = Table.from_pandas(df)

## Generate the Grism Dispersion Lengths Plot

In [ ]:
f, ax1 = plt.subplots(1,figsize=(15, 20))
ms = 14
for i in np.arange(0,len(table)):
    ax1.plot([table[i]['lambda1'],table[i]['lambda2']], [int(i),int(i)], 'k', lw=3) 
    ax1.plot([table2[i]['lambda1']*2,table2[i]['lambda2']*2], [int(i),int(i)], 'k', alpha=0.3, lw=3) 
    ax1.text(1.5, int(i), table[i]['filter'], va='center', fontsize=22)
    ax1.plot(3.9, int(i), 'kx', ms=ms)

ax1.plot(3.9, 4, 'kx', ms=ms)
ax1.plot(3.9, 3, 'kx', ms=ms)
ax1.plot(3.9, 2, 'kx', ms=ms)
ax1.plot(3.9, 1, 'kx', ms=ms)
ax1.plot(3.9, 0, 'kx', ms=ms)

ax1.text(4.66, 5., 'detector', va='center', fontsize=22,color='red')
ax1.text(4.5, 5.35, '(2040 pixels)', va='center', fontsize=22,color='red')
ax1.text(3.0, -0.8, '-x (Row)', va='center', fontsize=22,color='red')
ax1.text(3.4, -1.5, 'source position', va='center', fontsize=22,color='black')

xticks = ax1.xaxis.get_major_ticks()
xticks[0].label1.set_visible(False)

ax1.set_xlim(1.25, 5.5)

ax1.xaxis.tick_top()
ax1.xaxis.set_label_position('top') 
ax1.xaxis.set_ticks_position('both')

ax1.tick_params(labelsize=20)
plt.setp(ax1.get_yticklines(),visible=False)
ax1.tick_params('x', length=10, width=0.5,direction='in',which='major')

ax1.add_patch(patches.Rectangle((2.2, 3.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, 2.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, 1.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, 0.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, -0.25),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, 4.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 5.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 6.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 7.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 8.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 9.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 10.75),2, 0.5,fill=False,color='red',lw=4))

ax1.arrow(3.0, -0.45, 0.44, 0, head_width=0.22, head_length=0.1, fc='r', ec='r')
ax1.arrow(3.9, -1.15, 0, 0.6,head_width=0.1,head_length=0.2, fc='k', ec='k')

ax1.annotate('$1^{st}$ order',
            xy=(1.44,-2.0), xycoords='data',
            xytext=(80, 0), textcoords='offset points',
            arrowprops=dict(arrowstyle='-', ls='solid',facecolor='black',lw=4,shrinkB=3),
            horizontalalignment='center', verticalalignment='center',fontsize=18)    
ax1.annotate('$2^{nd}$ order',
            xy=(1.44,-1.6), xycoords='data',
            xytext=(81, 0), textcoords='offset points',
            arrowprops=dict(arrowstyle='-', ls='solid',facecolor='black',alpha=0.3,lw=4,shrinkB=3),
            horizontalalignment='center', verticalalignment='center',fontsize=18) 

ax1.add_patch(patches.Rectangle((1.33, -2.3),0.9, 1,fill=False,color='grey',lw=1))

ax1.set_yticklabels([])
plt.gca().invert_yaxis()
ax1.set_ylim(11.7, -2.5)

f.text(0.5, 0.925,'First-Order Wavelength (microns)', ha='center', fontsize=20)

ax1.xaxis.get_major_ticks()[1].draw = lambda *args:None
ax1.xaxis.get_major_ticks()[-1].draw = lambda *args:None

## Issues

* None

## About this Notebook
**Authors:**   
Dan Coe & Alicia Canipe

**Updated On:**   
April 10, 2019